In [2]:
import pandas as pd

In [3]:
# 다음 네이버 기사 불러오기
daum_news_data_tokenized= pd.read_csv("./daum_news_tokenized.csv")
naver_news_tokenized= pd.read_csv("./naver_news_tokenized.csv")

In [4]:
# 2024. 06. 01 ~ 2024. 09. 01 데이터만 추출

df_daum=daum_news_data_tokenized
df_naver=naver_news_tokenized

def convert_date_naver(date):
    date = date.split('. ')[0]
    if date == 'Null':
        return np.nan
    return pd.to_datetime(date, format='%Y.%m.%d').date()
df_naver['publication_date'] = df_naver['publication_date'].apply(convert_date_naver)
def convert_date_daum(date):
    date = date.split('. ')[:-1]
    date = '.'.join(date)
    if date == 'Null':
        return np.nan
    return pd.to_datetime(date, format='%Y.%m.%d').date()
df_daum['publication_date'] = df_daum['publication_date'].apply(convert_date_daum)
df_naver.dropna(subset=['title', 'content', 'publication_date'], inplace=True)
df_daum.dropna(subset=['title', 'content', 'publication_date'], inplace=True)
start_date = pd.to_datetime('2024-06-01').date()
end_date = pd.to_datetime('2024-09-01').date()
filtered_naver = df_naver[(df_naver['publication_date'] >= start_date) & (df_naver['publication_date'] <= end_date)]
filtered_daum = df_daum[(df_daum['publication_date'] >= start_date) & (df_daum['publication_date'] <= end_date)]

In [5]:
# 다음과 네이버 기사 합치기
daum_naver= pd.concat([filtered_daum,filtered_naver])

# 공백 문자 및 빈 문자열을 모두 NaN으로 변환
daum_naver['content'] = daum_naver['content'].str.strip().replace(r'^\s*$', float('NaN'), regex=True)
daum_naver['title'] = daum_naver['title'].str.strip().replace(r'^\s*$', float('NaN'), regex=True)

# NaN 제거
daum_naver = daum_naver.dropna(subset=['title', 'content'])
print(len(daum_naver))

# 날짜순 정렬
daum_naver=daum_naver.sort_values('publication_date')

# 인덱스 재설정
daum_naver = daum_naver.reset_index(drop=True)

80474


In [6]:
# 제목 데이터를 하나의 문자열로 합침
merge_daum_naver= ' '.join(daum_naver['title'])

# DataFrame 생성 (하나의 행으로 저장됨)
merge_daum_naver= pd.DataFrame({'data': [merge_daum_naver]})

# CSV 파일로 저장
merge_daum_naver.to_csv("./merge_naver_news_.csv", index=False)
merge_daum_naver



,data
0,강 욱 논란 노동 시 문제 나라 미 복 귀 전 공 수 련 포기 해외 진출 동 의원 ...


In [7]:
# 내용 최빈값 3개 추출

from collections import Counter
import pandas as pd

# naver
# 'content_3' 열 추가
daum_naver['content_3'] = ""
count=0
count_list=[]

# 기사 내용 단어 카운트
for i in range(len(daum_naver['content'])):
    text = daum_naver['content'][i]

    # 텍스트를 공백 기준으로 토큰화
    words = text.split()

    # 단어 빈도 계산
    word_counts = Counter(words)

    # 상위 3개의 단어 빈도 추출
    common_words = word_counts.most_common(3)

    # 딕셔너리로 변환
    word_count_dict = {word: count for word, count in common_words}

    # content_3 열에 딕셔너리 저장
    daum_naver.at[i, 'content_3']= word_count_dict

# CSV 파일로 저장
daum_naver.to_csv("./daum_naver3_3months.csv")

In [ ]:
# 데이터 불러오기
daum_naver= pd.read_csv("./daum_naver3_3months.csv")

In [8]:
# 전체 제목 중 내용 최빈값이 몇번 나오는지 카운트


# naver
# 'daum_naver.iloc[0]'을 문자열로 변환
merge_string = str(merge_daum_naver["data"][0])

count=0
count_title_content={}
key=[]
# daum_naver['content_3']에서 상위 3개의 키 단어를 가져와서 count
for i in range(len(daum_naver['content_3'])):

    # 딕셔너리의 키를 리스트로 변환
    key_list = list(daum_naver['content_3'][i].keys())
    # 키 축적
    key+=key_list
# 키 중복 제거
unique_key=set(key)

# 전체 기사 제목에서 최빈값 등장횟수 구하기
for key_word in unique_key:

    # merge_string에서 해당 단어가 몇 번 나오는지 카운트 (문자열에서 직접 카운트)
    word_count_in_merge = merge_string.count(key_word)
    print(f"{key_word}: {word_count_in_merge}")
    count_title_content[key_word]=word_count_in_merge

# 내림차순 정렬
sorted_Frequency = sorted(count_title_content.items(), key= lambda item:item[1], reverse=True)
print(sorted_Frequency)

# 칼럼명 지정
Frequency=pd.DataFrame(sorted_Frequency, columns=['word', 'frequency'])

# 'frequency' 열의 글자 수가 2개 이상인 행만 필터링
Frequency = Frequency[Frequency['word'].astype(str).str.len() >= 2]

# 등장횟수별 정렬
Frequency=Frequency.sort_values('frequency',ascending=False)

# 인덱스 재설정
Frequency = Frequency.reset_index(drop=True)

# 최빈값 갯수
print(len(Frequency))
# 저장
Frequency.to_csv("./Frequency_naver_3months.csv", index=False)

카드: 243
대비: 344
최고: 546
대응: 537
스크린: 36
스포츠: 431
툰: 710
률: 106
드래: 140
포인트: 146
우수: 383
언론: 71
영국: 49
도로: 74
통합: 450
시스템: 985
똥: 72
광주: 59
하이: 424
비상: 213
닛: 0
여성: 155
악: 576
번호: 175
방송: 812
시청: 145
산업단지: 7
초: 1797
병: 1622
줌: 37
올스타: 34
국세청: 5
기사: 82
아이템: 88
릉: 73
직: 1889
겟: 9
통화: 100
일부: 102
보드: 101
경남: 30
영어: 57
빔: 35
증가: 486
예산: 323
징계: 16
랜드: 539
코: 2956
안드로이드: 21
쑤: 1
항: 888
엔지니어링: 35
의료: 411
쏘: 176
수원: 60
루: 1317
법인: 231
컴퓨터: 51
팰: 0
흡연: 6
증권: 162
십: 718
포트: 241
웍: 234
필름: 61
매출: 1100
고양: 48
불법: 254
물류: 53
부장: 86
혜: 599
태평양: 30
봄: 71
푸드: 57
수출: 217
생산: 363
벽: 203
더: 314
예약: 213
결: 3045
아트: 109
베이스: 80
안보: 104
모델: 507
과: 4053
구단: 40
화재: 61
슈퍼: 320
직원: 394
것: 1461
제작: 258
살: 129
충: 815
서울대: 194
니스: 248
연: 5606
영등포: 0
깃: 46
데: 3845
상처: 21
딘: 44
의원: 246
전기: 428
나: 2304
사례: 141
브라운: 21
밖: 36
중소: 201
운: 1861
노트: 152
채널: 250
젝: 407
아침: 83
샌: 22
실: 3018
대구: 80
영업: 1113
공시: 63
수수료: 133
돌: 1288
마켓: 260
사무소: 8
강릉: 17
랩: 795
촬영: 87
배출: 62
사태: 326
추천: 185
접근: 98
현황: 36
검증: 224
복지: 181
카메라: 11

In [9]:
len(unique_key)

1809